### SageMaker Counterfactual prediction

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)
step_func_role = "arn:aws:iam::{}:role/AmazonSageMaker-StepFunctionsWorkflowExecutionRole".format(account_id)

In [2]:
version_tag="202304091951"
pytorch_custom_image_name=f"large-scale-ptm-ppi:gpu-{version_tag}"
instance_type = "ml.g4dn.2xlarge"  #ml.g4dn.2xlarge
instance_count = 1



In [3]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

In [4]:
bucket = "aegovan-data"

In [5]:

dataset = { 
    "yelp" :  "s3://aegovan-data/yelp_polarity/test.csv",
    "semeval": "s3://aegovan-data/semeval2017taskb/SemEval2017-task4-test.subtask-BD.english.txt",
    "amazon":  "s3://aegovan-data/amazon_reviews_polarity/test.csv",


}


dataset_type="amazon"
eval_file = dataset[dataset_type]



In [6]:
import datetime
date_fmt = datetime.datetime.today().strftime("%Y%m%d%H%M")

In [7]:
training_job = "counterfact-imdb-23-0-0-2023-04-10-04-21-55-243"
#training_job  = "counterfact-imdb-0-0-0-2023-04-10-03-38-52-252"
#training_job  = "counterfact-imdb-9-0-0-2023-04-10-02-21-27-455"
#training_job = "counterfact-imdb-0-0-0-2023-04-09-21-10-10-142"
#training_job =  "counterfact-imdb-original-2023-04-09-19-03-45-522" # .90 adv
#training_job = "counterfact-imdb-simple-2023-04-03-01-35-49-441" # .25 adv
#training_job = "counterfact-imdb-simple-2023-04-09-02-17-57-367"  # .10 adv
s3_model_path = f"s3://aegovan-data/sagemakerresults/{training_job}/output/model.tar.gz"
s3_output_predictions = "s3://aegovan-data/counterfactual_{}/predictions_{}/{}".format(dataset_type,training_job,date_fmt)

training_job_prefix="imdb202304171350"

In [8]:
prediction_jobs = []


# for dataset_type, dataset_uri in dataset.items():
#     for i in range(1,5):
#         for adv in [0, 9, 19, 28, 92]:
#             training_job_name= f"{training_job_prefix}-{adv:02d}-00-00-{i:02d}"
#             item = {"job_name":f"{training_job_prefix}-{date_fmt}-{dataset_type}-{adv:02d}-{i:02d}",
#                         "model_uri" :f"s3://aegovan-data/sagemakerresults/{training_job_name}/output/model.tar.gz",
#                         "s3_output_predictions": "s3://aegovan-data/counterfactual_{}/predictions_{}/{}".format(dataset_type,training_job_name,date_fmt),
#                         "input_dataset" : dataset_uri,
#                         "dataset_type" : dataset_type
#                          }
#             prediction_jobs.append(item)

# training_jobs = ["imdb202304171350-92-00-00-01",
# "imdb202304171350-29-00-00-01",
# "imdb202304171350-19-00-00-01",
# "imdb202304171350-10-00-00-01",
# "imdb202304171350-00-00-00-01",
# "imdb202304171350-92-00-00-05",
# "imdb202304171350-29-00-00-05",
# "imdb202304171350-19-01-00-05",
# "imdb202304171350-09-00-00-05",
# "imdb202304171350-00-00-00-05",
# "imdb202304171350-92-00-00-04",
# "imdb202304171350-28-00-00-04",
# "imdb202304171350-18-01-00-04",
# "imdb202304171350-09-00-00-04",
# "imdb202304171350-00-00-00-04",
# "imdb202304171350-92-00-00-03",
# "imdb202304171350-29-00-00-03",
# "imdb202304171350-19-00-00-03",
# "imdb202304171350-09-01-00-03",
# "imdb202304171350-00-00-00-03",
# "imdb202304171350-92-00-00-02",
# "imdb202304171350-29-00-00-02",
# "imdb202304171350-19-00-00-02",
# "imdb202304171350-09-00-00-02"]


  
#     "imdb202304212023-92-00-00-05",
# "imdb202304212023-28-00-00-05",
# "imdb202304212023-19-00-00-05",
# "imdb202304212023-09-00-00-05",
# "imdb202304212023-00-00-00-05",
# "imdb202304212023-92-00-00-04",
# "imdb202304212023-29-00-00-04",
# "imdb202304212023-19-00-00-04",
# "imdb202304212023-09-00-00-04",
# "imdb202304212023-00-00-00-04",
# "imdb202304212023-93-00-00-01",
# "imdb202304212023-92-00-00-01",
# "imdb202304212023-29-00-00-01",
# "imdb202304212023-19-00-00-01",
# "imdb202304212023-09-00-00-01",
# "imdb202304212023-00-00-00-01",
# "imdb202304212023-93-00-00-03",
# "imdb202304212023-28-00-00-03",
# "imdb202304212023-19-00-00-03",
# "imdb202304212023-09-01-00-03",
# "imdb202304212023-00-01-00-03",
# "imdb202304212023-92-00-00-02",
# "imdb202304212023-29-00-00-02",
# "imdb202304212023-19-00-00-02",
# "imdb202304212023-09-00-00-02",
# "imdb202304212023-00-00-00-02"

training_jobs = [
"imdb202306041410-88-00-00-05",
"imdb202306041410-26-00-00-05",
"imdb202306041410-18-00-00-05",
"imdb202306041410-09-01-00-05",
"imdb202306041410-00-00-00-05",
"imdb202306041410-92-00-00-04",
"imdb202306041410-28-00-00-04",
"imdb202306041410-18-00-00-04",
"imdb202306041410-09-00-00-04",
"imdb202306041410-00-00-00-04",
"imdb202306041410-87-00-00-03",
"imdb202306041410-28-00-00-03",
"imdb202306041410-20-00-00-03",
"imdb202306041410-09-00-00-03",
"imdb202306041410-00-00-00-03",
"imdb202306041410-90-00-00-02",
"imdb202306041410-27-00-00-02",
"imdb202306041410-18-00-00-02",
"imdb202306041410-08-00-00-02",
"imdb202306041410-00-00-00-02",
"imdb202306041410-88-00-00-01",
"imdb202306041410-29-00-00-01",
"imdb202306041410-17-00-00-01",
"imdb202306041410-09-00-00-01",
"imdb202306041410-00-00-00-01"
]

for dataset_type, dataset_uri in dataset.items():
    for training_job_name in training_jobs:

        item = {"job_name":f"{training_job_name}-{date_fmt}-{dataset_type}",
                    "model_uri" :f"s3://aegovan-data/sagemakerresults/{training_job_name}/output/model.tar.gz",
                    "s3_output_predictions": "s3://aegovan-data/counterfactual_{}/predictions_{}/{}".format(dataset_type,training_job_name,date_fmt),
                    "input_dataset" : dataset_uri,
                    "dataset_type" : dataset_type
                     }
        prediction_jobs.append(item)

In [9]:
prediction_jobs

[{'job_name': 'imdb202306041410-88-00-00-05-202306041556-yelp',
  'model_uri': 's3://aegovan-data/sagemakerresults/imdb202306041410-88-00-00-05/output/model.tar.gz',
  's3_output_predictions': 's3://aegovan-data/counterfactual_yelp/predictions_imdb202306041410-88-00-00-05/202306041556',
  'input_dataset': 's3://aegovan-data/yelp_polarity/test.csv',
  'dataset_type': 'yelp'},
 {'job_name': 'imdb202306041410-26-00-00-05-202306041556-yelp',
  'model_uri': 's3://aegovan-data/sagemakerresults/imdb202306041410-26-00-00-05/output/model.tar.gz',
  's3_output_predictions': 's3://aegovan-data/counterfactual_yelp/predictions_imdb202306041410-26-00-00-05/202306041556',
  'input_dataset': 's3://aegovan-data/yelp_polarity/test.csv',
  'dataset_type': 'yelp'},
 {'job_name': 'imdb202306041410-18-00-00-05-202306041556-yelp',
  'model_uri': 's3://aegovan-data/sagemakerresults/imdb202306041410-18-00-00-05/output/model.tar.gz',
  's3_output_predictions': 's3://aegovan-data/counterfactual_yelp/predictions_

### Run  prediction

In [10]:
#s3_output_predictions = "s3://aegovan-data/pubmed_asbtract/predictions_largescale_{}_{}/".format(job_prefix,date_fmt)
s3_data_type="S3Prefix"
usefilter=0
filter_threshold_std=1.0

s3_input_models = s3_model_path
s3_input_vocab = "s3://aegovan-data/pretrained_models/bert-base-uncased/"

In [11]:
from sagemaker.network import NetworkConfig
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow
import random

processing_steps = []

PREPROCESSING_SCRIPT_LOCATION = "../src/inference/main_predict_sentiment_polarity.py"

s3_input_code = sagemaker_session.upload_data(
    PREPROCESSING_SCRIPT_LOCATION,
    bucket=sagemaker_session.default_bucket(),
    key_prefix="code/imdb/code",
)

for j in prediction_jobs:

    s3_model_path =  j["model_uri"]
    s3_output_predictions =j["s3_output_predictions"]
    s3_input_models = s3_model_path
    eval_file =  j["input_dataset"]
    s3_input_data = eval_file
    dataset_type = j["dataset_type"]
    
    script_processor = ScriptProcessor(image_uri=docker_repo,
                                           command=["python"],
                                           env={'mode': 'python', 'PYTHONPATH':'/opt/ml/code'},
                                           role=role,
                                           instance_type=instance_type,
                                           instance_count=1,
                                           max_runtime_in_seconds= 5 * 24 * 60 * 60,
                                           volume_size_in_gb = 200,
                                           network_config=NetworkConfig(enable_network_isolation=False),
                                           base_job_name =f"{dataset_type}-inference"
                                           )


    sm_local_input_models = "/opt/ml/processing/input/data/models"
    sm_local_input_data = "/opt/ml/processing/input/data/jsondata"
    sm_local_input_vocab = "/opt/ml/processing/input/data/vocab"


    sm_local_output = "/opt/ml/processing/output"

    
    step_processing = sagemaker.ProcessingStep(f"imdb-{j['job_name']}", 
            script_processor, 
            j["job_name"], 
            container_entrypoint=['python', '/opt/ml/processing/input/code/main_predict_sentiment_polarity.py'],
                
            container_arguments=[
                f"{sm_local_input_data}/{eval_file.split('/')[-1]}",
                sm_local_input_models,
                sm_local_output,
                dataset_type,
                "--tokenisor_data_dir", sm_local_input_vocab,           
                "--batch", "32"
            ],


            inputs=[
                ProcessingInput(
                                source=s3_input_code,
                                destination="/opt/ml/processing/input/code",
                                input_name="code",
                            ),
                ProcessingInput(
                        source=s3_input_data,
                        s3_data_type = s3_data_type,
                        destination=sm_local_input_data,
                        input_name="input-1",
                        s3_data_distribution_type="ShardedByS3Key"),

                ProcessingInput(
                        source=s3_input_models,
                        input_name="input-model",
                        destination=sm_local_input_models,
                        s3_data_distribution_type="FullyReplicated"),

                ProcessingInput(
                        source=s3_input_vocab,
                        input_name="input-vocab",
                        destination=sm_local_input_vocab,
                        s3_data_distribution_type="FullyReplicated")
                
                
                ],


            outputs=[ProcessingOutput(
                    source=sm_local_output, 
                    destination=s3_output_predictions,
                    output_name='predictions')]
        )

    
    processing_steps.append(step_processing)



parallel_steps = []
max_parallel =10
#parallel_steps.append(states.Wait(f"wait", seconds=60*60*3))
for i in range(0, len(processing_steps), max_parallel):
    p = states.Parallel(f"predict-p-{i}")
    for si, s in enumerate(processing_steps[i: i+max_parallel]):
        w = (si+1)*30
        p.add_branch(Chain([states.Wait(f"wait-{i}-{w}", seconds=w) , s]))
    parallel_steps.append(p)

    
basic_path = Chain(parallel_steps)

basic_workflow = Workflow(
    name=f"imdb-infer-{date_fmt}", definition=basic_path, role=step_func_role
)

# Render the workflow
basic_workflow.render_graph()


In [12]:
basic_workflow.create()

'arn:aws:states:us-east-2:324346001917:stateMachine:imdb-infer-202306041556'

In [13]:
basic_workflow.execute()

Execution(execution_arn='arn:aws:states:us-east-2:324346001917:execution:imdb-infer-202306041556:d2110986-2c51-4ff1-8529-c834f903b397', name=None, status=<ExecutionStatus.Running: 'RUNNING'>, start_date=datetime.datetime(2023, 6, 4, 15, 57, 10, 941000, tzinfo=tzlocal()))